In [ ]:
from flathub_python_api import ApiClient, Configuration, CollectionApi, AppsIndex
from typing import Optional
import json

In [6]:
# Initialize the API client
client = ApiClient(configuration=Configuration(host="https://flathub.org/api/v2/"))

In [7]:
collection_api = CollectionApi(client)

In [14]:
categories = json.loads(collection_api.get_categories_collection_category_get_without_preload_content().data)

In [15]:
categories

['audiovideo',
 'development',
 'education',
 'game',
 'graphics',
 'network',
 'office',
 'science',
 'system',
 'utility']

In [16]:
def get_all_apps_in_category(collection_api: CollectionApi, category: str, limit: Optional[int] = None):
    """
    Fetch all apps in a specific category.
    
    :param category: The category to fetch apps from.
    :param limit: Optional limit on the number of apps to fetch.
    :return: List of apps in the specified category.
    """
    apps = []
    page = 1
    
    while True:
        print(f"Fetching page {page} for category '{category}'...")
        response = collection_api.get_category_collection_category_category_get(
            category=category, page=page, per_page=100
        )
        
        apps.extend(response.hits)
        if page == response.total_pages:
            break
        if limit and len(apps) >= limit:
            print(f"Reached limit of {limit} apps.")
            break

        page += 1
    return apps[:limit] if limit else apps


In [ ]:
apps: list[AppsIndex] = get_all_apps_in_category(collection_api, "audiovideo")  # Example usage to fetch 10 apps in the "audiovideo" category

Fetching page 1 for category 'audiovideo'...
Fetching page 2 for category 'audiovideo'...
Fetching page 3 for category 'audiovideo'...
Fetching page 4 for category 'audiovideo'...


In [19]:
len(apps)

321

In [ ]:
for app in apps:
    print(f"{app.name:<25} ID: {app.id:<50} Summary: {app.summary}")

Spotify                   ID: com_spotify_Client                                , Summary: Online music streaming service
VLC                       ID: org_videolan_VLC                                  , Summary: VLC media player, the open-source multimedia player
OBS Studio                ID: com_obsproject_Studio                             , Summary: Live stream and record videos
FreeTube                  ID: io_freetubeapp_FreeTube                           , Summary: An Open Source YouTube app for privacy
Haruna                    ID: org_kde_haruna                                    , Summary: Media player
Kdenlive                  ID: org_kde_kdenlive                                  , Summary: Video editor
Stremio                   ID: com_stremio_Stremio                               , Summary: Freedom to Stream
HandBrake                 ID: fr_handbrake_ghb                                  , Summary: Video Transcoder
Easy Effects              ID: com_github_wwmm_easyeffects  